In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import MDAnalysis as mda
import nglview as nv

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [4]:
# Carga las dinamicas y selecciona las proteinas
u = mda.Universe("../data/raw/Prot_Lig_ions.prmtop", "../data/raw/TY_DDAT_ions.nc")
protein = u.select_atoms("protein")

In [5]:
# Visualiza la dinamics
w = nv.show_mdanalysis(protein)
w

NGLWidget(max_frame=99999)